# TM10007 Assignment template

In [1]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/jveenland/tm10007_ml.git

# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn

# Preprocessing
from sklearn import model_selection

# Classifiers and kernels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel
from sklearn.model_selection import GridSearchCV, StratifiedKFold 
from sklearn import metrics
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.preprocessing import LabelEncoder

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [2]:
from worclipo.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')
print(data)

The number of samples: 115
The number of columns: 494
                  label  PREDICT_original_sf_compactness_avg_2.5D  \
ID                                                                  
Lipo-001_0  liposarcoma                                  0.368434   
Lipo-002_0  liposarcoma                                  0.660571   
Lipo-003_0       lipoma                                  0.365191   
Lipo-004_0  liposarcoma                                  0.372210   
Lipo-005_0       lipoma                                  0.369120   
...                 ...                                       ...   
Lipo-111_0       lipoma                                  0.244813   
Lipo-112_0  liposarcoma                                  0.197353   
Lipo-113_0  liposarcoma                                  0.307562   
Lipo-114_0       lipoma                                  0.577333   
Lipo-115_0  liposarcoma                                  0.635282   

            PREDICT_original_sf_compactness_std_

In [3]:
"""Splitting the data into a test and training dataset: outer cross-validation"""
#Outer cross-validation
#Split the dataset in train and test part
data_train, data_test = model_selection.train_test_split(data, test_size=0.20)
print(f'The number of samples train: {len(data_train.index)}')
print(f'The number of columns train: {len(data_train.columns)}')
print(f'The number of samples test: {len(data_test.index)}')
print(f'The number of columns test: {len(data_test.columns)}')

The number of samples train: 92
The number of columns train: 494
The number of samples test: 23
The number of columns test: 494


In [4]:
# Outliers detecteren en vervangen
from outliers import outlier_detection
data_train, total_outliers = outlier_detection(data_train)
print("\nTotaal aantal gecapte outliers train data:", total_outliers)

data_test, total_outliers = outlier_detection(data_test)
print("\nTotaal aantal gecapte outliers test data:", total_outliers)

c:\Users\robin\Documents\Technical medicine\TM10007 Machine Learning\Group assignment\TM10007_ML_Project_group_9\outliers.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(data[column])
c:\Users\robin\Documents\Technical medicine\TM10007 Machine Learning\Group assignment\TM10007_ML_Project_group_9\outliers.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[26.22771825 26.22771825 26.22771825 26.22771825 26.22771825 26.22771825
 26.22771825]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[outliers,column] = np.clip(data.loc[outliers,column], mean_value - drempel * std_value, mean_value + drempel * std_value)
c:\Users\robin\Documents\Technical medicine\TM10007 Machine Learning\Group assignment\TM10007_ML_Proj


Totaal aantal gecapte outliers train data: 702


c:\Users\robin\Documents\Technical medicine\TM10007 Machine Learning\Group assignment\TM10007_ML_Project_group_9\outliers.py:9: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  z_scores = zscore(data[column])
c:\Users\robin\Documents\Technical medicine\TM10007 Machine Learning\Group assignment\TM10007_ML_Project_group_9\outliers.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[31.60544092]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data.loc[outliers,column] = np.clip(data.loc[outliers,column], mean_value - drempel * std_value, mean_value + drempel * std_value)
c:\Users\robin\Documents\Technical medicine\TM10007 Machine Learning\Group assignment\TM10007_ML_Project_group_9\outliers.py:9: RuntimeWarning: Precision loss occurred in mom


Totaal aantal gecapte outliers test data: 98


In [6]:
from prepro_data import processing_data_scaling
#This data is scaled and a variance and correlation threshold are applied
data_scaled, df_label, df_processed= processing_data_scaling(data_train)
print(data_scaled)

                  label         0         1         2         3         4  \
ID                                                                          
Lipo-023_0       lipoma  2.743640  2.416763 -0.591909 -0.097447 -0.288273   
Lipo-039_0       lipoma -0.906803 -0.888038 -0.627873 -0.497345 -0.602633   
Lipo-030_0  liposarcoma  0.677908 -0.144497  1.524376 -0.430130 -0.094771   
Lipo-080_0       lipoma  0.023619  0.775574  0.093168 -0.477254 -0.605137   
Lipo-079_0  liposarcoma  2.749149  3.258223  0.339542  3.807243  3.153359   
...                 ...       ...       ...       ...       ...       ...   
Lipo-013_0  liposarcoma -0.326863 -0.557398 -0.017888 -0.436727 -0.245828   
Lipo-109_0       lipoma -0.505102 -0.751764 -0.685472 -0.421579 -0.381757   
Lipo-010_0       lipoma -0.637994  0.163488 -0.615554 -0.498954 -0.227294   
Lipo-005_0       lipoma  1.936436  2.605551 -0.706616 -0.255357 -0.397384   
Lipo-017_0       lipoma -0.582703 -0.688407 -0.476663 -0.414437 -0.563626   

In [7]:
from prepro_data import processing_data_pca
#This data is scaled, a variance and correlation threshold are applied, 
# and PCA brought back the number of features to 36
data_pca, df_label, df_processed= processing_data_pca(data_train)
print(data_pca)

                  label          0         1         2         3         4  \
ID                                                                           
Lipo-023_0       lipoma   8.564778  6.330343 -2.184832  2.080646  0.953118   
Lipo-039_0       lipoma  -3.923212 -0.123592 -0.202061  0.323418  0.684116   
Lipo-030_0  liposarcoma   9.786830 -3.326241  7.364727 -1.748676 -3.832685   
Lipo-080_0       lipoma  -1.526176 -4.414299  1.369508 -4.213900 -1.841516   
Lipo-079_0  liposarcoma  10.427448  2.860989 -4.881146  0.333194 -0.176703   
...                 ...        ...       ...       ...       ...       ...   
Lipo-013_0  liposarcoma   0.744386 -1.512615 -0.986654  5.098314  0.784057   
Lipo-109_0       lipoma   0.356610 -2.431373 -2.907800 -1.384983  1.615274   
Lipo-010_0       lipoma  -0.610163 -3.503863  0.388584  2.699697 -0.443428   
Lipo-005_0       lipoma   7.622545  3.787266 -4.663857 -1.841782 -0.735235   
Lipo-017_0       lipoma  -1.291572 -1.297841  0.854477 -0.219022

In [8]:
from prepro_data import processing_data_rfecv
#This data is scaled, a variance and correlation threshold are applied,
# and RFECV brought back the number of features to 61 (takes a bit longer to run)
data_rf, df_label, df_processed= processing_data_rfecv(data_train)
print(data_rf)

                 0         2         3         4         6         14   \
ID                                                                       
Lipo-023_0  2.743640 -0.591909 -0.097447 -0.288273  0.755502 -0.649119   
Lipo-039_0 -0.906803 -0.627873 -0.497345 -0.602633  0.375392 -0.554547   
Lipo-030_0  0.677908  1.524376 -0.430130 -0.094771  2.514622  0.557397   
Lipo-080_0  0.023619  0.093168 -0.477254 -0.605137 -1.438785  3.180472   
Lipo-079_0  2.749149  0.339542  3.807243  3.153359  1.708361 -0.250857   
...              ...       ...       ...       ...       ...       ...   
Lipo-013_0 -0.326863 -0.017888 -0.436727 -0.245828 -0.924287 -0.988486   
Lipo-109_0 -0.505102 -0.685472 -0.421579 -0.381757 -0.196961  0.414336   
Lipo-010_0 -0.637994 -0.615554 -0.498954 -0.227294 -1.069276 -0.687566   
Lipo-005_0  1.936436 -0.706616 -0.255357 -0.397384  0.240721  0.092646   
Lipo-017_0 -0.582703 -0.476663 -0.414437 -0.563626 -0.592271 -1.041437   

                 16        17        

In [ ]:
"""Support Vector Machine"""
# Split data into features and labels
X = data_pca.drop(data_pca.columns[[0, 1]], axis=1)  # Features
le = LabelEncoder()
Y = le.fit_transform(df_label['label'])  # Now Y is numeric (0 or 1)

# Set up the parameter grid for GridSearchCV
param_grid = [
    {'kernel': ['linear'], 'C': [0.1, 1, 10]},
    {'kernel': ['rbf'], 'C': [0.1, 1, 10], 'gamma': ['scale', 'auto', 0.1, 1]},
    {'kernel': ['poly'], 'C': [0.1, 1, 10], 'degree': [3, 4, 5], 'gamma': ['scale', 'auto', 0.1, 1], 'coef0': [0.0, 0.1, 0.5]},
]

# Set up the SVM classifier
svm_classifier = SVC(random_state=42, probability=True)  # Enable probability for AUC calculation

# Set up inner cross-validation for hyperparameter tuning
inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Set up GridSearchCV to search for the best hyperparameters
grid_search = GridSearchCV(
    estimator=svm_classifier, 
    param_grid=param_grid, 
    cv=inner_cv, 
    n_jobs=-1, 
    verbose=1, 
    scoring=['roc_auc', 'f1'],  # Optimize for both AUC and F1-score
    refit='roc_auc'  # Select the best model based on AUC
)

# Perform the grid search on the training data to find the best hyperparameters
grid_search.fit(X, Y)

# Get the best model (with the best hyperparameters)
best_model = grid_search.best_estimator_

# Make predictions to evaluate performance
y_pred = best_model.predict(X)
y_pred_proba = best_model.predict_proba(X)[:, 1]  # Probability estimates for AUC

# Convert categorical labels to numerical labels (0 or 1)
le = LabelEncoder()
Y = le.fit_transform(Y)  # 'lipoma' -> 0, 'liposarcoma' -> 1

# Ensure predictions are also numerical
y_pred = best_model.predict(X)
y_pred = le.transform(y_pred)  # Convert categorical predictions ('lipoma', 'liposarcoma') to 0 and 1

# Calculate evaluation metrics
auc_score = roc_auc_score(Y, y_pred_proba)
f1 = f1_score(Y, y_pred)

# Print results
print(f"Best hyperparameters: {grid_search.best_params_}")
print(f"Training AUC: {auc_score:.4f}")
print(f"Training F1-score: {f1:.4f}")

Fitting 5 folds for each of 123 candidates, totalling 615 fits
Best hyperparameters: {'C': 1, 'coef0': 0.0, 'degree': 5, 'gamma': 'scale', 'kernel': 'poly'}
Training AUC: 1.0000
Training F1-score: 0.7353


In [ ]:
"""Random Forest"""

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(df_processed, df_label) #Input is nu dataframe met alleen de features en dataframe met alleen de labels
y_val_pred = rf_classifier.predict(data_val)
val_accuracy = accuracy_score(df_label_val, y_val_pred)
print(f'Validation Accuracy without hyperparameters: {val_accuracy * 100:.2f}%')

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42), param_grid=param_grid, cv=5)
grid_search.fit(data_train, df_label)
print("Best Parameters:", grid_search.best_params_)
best_rf = grid_search.best_estimator_
y_val_pred_best = best_rf.predict(data_val)
val_accuracy_best = accuracy_score(df_label_val, y_val_pred_best)
print(f'Validation Accuracy (Best Model): {val_accuracy_best * 100:.2f}%')